In [1]:
from scipy.signal import butter,lfilter
from scipy.ndimage.filters import gaussian_filter1d
from matplotlib.pyplot import mlab
from GEMsortfunc_for_multi import GEMsort
from data_func import get_chunk, cut
from numpy import apply_along_axis as apply
from scipy.signal import fftconvolve
from scipy import signal
from functools import reduce
from numpy.linalg import svd
from numpy import *
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial

import numpy.matlib
import scipy, os
import matplotlib.pyplot as plt
import xml.etree.ElementTree
import pandas as pd
import struct
import copy
import numpy as np
import seaborn as sns
import time

In [2]:
mm = np.memmap('/.../data/experiment1_101-1_0.dat', dtype=np.int16,mode='r')

channels = 384
start = 0
end = 60
sampling_rate = 30000
T0 = 0
data = get_chunk(mm,start,end,channels,sampling_rate)

# sf = 30000 #sampling frequency
# T = data.shape[1] 
# dur_sec = T/sf 
# dur_min = dur_sec/60
# print(dur_min)

In [3]:
#common average referencing and filtering

ch_num = channels
med = np.median(data[:,T0:T],axis=0)
data_newm = np.zeros((data.shape[0],data.shape[1])) 

for i in range(384):
     data_newm[i,T0:T] = data[i,T0:T] - med

data_raw2 = copy.deepcopy(data_newm) 

for i in range(ch_num):
    #print(i)
    #i=ID_channels[0]
    data_raw = copy.deepcopy(data_raw2[i,:])

    def filter_data(data_raw, low, high, sf, order=2):
        nyq = sf/2
        low = low/nyq
        high = high/nyq
        b, a = butter(order, [low, high], btype='band')
        filtered_data = lfilter(b, a, data_raw)
        return filtered_data

    L = data.shape[1] #length of the data
    sf = 10000 
    filtered_data = filter_data(data_raw, low=100, high=800, sf=sf)

    data_newm[i,:] = filtered_data

data_new2 = data_newm        

NameError: name 'T' is not defined

In [ ]:
####### plot Neuropixels data heatmap #######
%matplotlib qt
npix_p3_reference_channels = np.array([36, 75, 112, 151, 188, 227, 264, 303, 340, 379])
ch_ref = npix_p3_reference_channels 
data_newm_good_channels= copy.deepcopy(data_newm)

for i, ch in enumerate(ch_ref):
    data_newm_good_channels [ch,:]= copy.deepcopy(data_newm[ch+1,:])
    
uniform_data = data_newm_good_channels[:, 352000:354000] #352000:354000
fig = plt.figure(figsize=(2*8, 20), dpi=150, facecolor='w', edgecolor='k')
# ax = sns.heatmap(uniform_data, cmap="rainbow", xticklabels=False, cbar=True)#, vmin=0.2, vmax=1)
ax = sns.heatmap(uniform_data, center=0, xticklabels=False, cbar=True)#, vmin=0.2, vmax=1)

plt.title('Neuropixels recording sample', fontsize=10)
plt.xlabel('Time (s)', fontsize=10)
plt.ylabel('Channel number', fontsize=10)
# plt.savefig(path + 'Neuropixels heatmap.png', dpi=300, format='png')

In [ ]:
####### Plot recordings of several channels together ####### 
%matplotlib qt
a = 120
b = 140
c = 352000
d = 352000 + 400
N = b - a #number of row windows
x = range(251000, 253000, 1)
# ymax=np.max(data_newm[145,:])
# ymin= np.min(data_newm[145,:])

fig = plt.figure()
gs = fig.add_gridspec(N, hspace=0)
axs = gs.subplots(sharex=True, sharey=True)
fig.suptitle('Nearby channels signals', fontsize=20)
plt.xlabel('Time (s)', fontsize=20)
# plt.ylabel('Channel number', fontsize=20)

for i in range(a, b):
    y1 = data_newm[i, c:d]
    i2 = i - a
    axs[i2].plot(y1)
    axs[i2].set(xticklabels=[])
    axs[i2].set(yticklabels=[])

# Duration of the plot     
dur_sec_diff = abs(d - c) / sf 
# dur_min1=dur_sec1/60
print(dur_sec_diff)
plt.show()   

In [ ]:
####### the main program for extracting the spikes of channels #######

Tdis = 150
start_time = time.time()
T1 = 82 
coef = 4.2

#sig64=np.zeros((X.shape[0],T1))
#p_all=np.zeros((X.shape[0],N)) 

X = data_new2 #just for one channel 
X[:,0:T0] = 0
GEMsort_all_times = []

for k in range (ch_num): 
    GEMsort_all_times .append([])

p_all = np.array([], dtype=object).reshape((1,0))
cut_ch = []

for k in range (ch_num):
    cut_ch .append([])

for i in range (ch_num):
    #print(i)
    x = X[i,:] 
    sigma = np.median(np.abs(x) / 0.6745)
    Tp = coef * sigma 
    p,_ = scipy.signal.find_peaks(np.abs(x), height=Tp, distance=Tdis) #peak(x,Tp,Tdis) #this function gives the places(times) that peaks occure
    GEMsort_all_times[i].append(p.astype('int')[:])
    #print(GEMsort_all_times[i])
    
    for j in range(np.array(p).shape[0]):
         cut_ch[i].append(cut(x,p[j],T1)[0]) 

    if p == []:
        p = [0] 
    y_arr = np.array([], dtype=np.int32)
    y = p
    y_arr = np.append(y_arr,y)
    p_all = np.append(p_all, 0)
    p_all[-1] = y_arr.astype(int) 
 

#print(p_all)
print("--- %s seconds ---" % (time.time() - start_time))

for i in range(ch_num):
    GEMsort_all_times[i] = np.array(GEMsort_all_times[i])[0]#just to have correct dimension


In [ ]:
####### Data reduction #######

start_time = time.time()
N = 5
ch_num = 384
all_ind_i = []
all_ind_j = []
xy = p_all[0]
cor_thr =  0.8

for s in range(round(ch_num/N)):
#     print('s =',s)
    s1 = s*N
    s2 = s1+N
    if s2 >= ch_num:
        s2 = ch_num-1
#     a = np.hstack(p_all[s1:s2]) 
#     rowp = a[None,:]
    #for k in range (np.array(rowp).shape[1]):
    xy = p_all[s1]

    for i in range (s1 + 1, s2 + 1): 
#         print('i=',i)
#         for j in range(i+1,s2):
#             print('j=',j)
        xy = np.intersect1d(p_all[i], xy)
    
    for j in range(N - 1):
        if (s1 + j + 1) < ch_num:
            for t in range(xy.shape[0]): 
                #print('s1+j+1=',s1+j+1)
                x_ind = np.where(xy[t] == p_all[s1])[0][0]
                y_ind = np.where(xy[t] == p_all[s1+j+1])[0][0]
                conf = scipy.stats.pearsonr(np.array(cut_ch)[s1][x_ind], np.array(cut_ch)[s1+j+1][y_ind])[0]

                if conf > cor_thr:
                    if np.max(np.array(cut_ch)[s1][x_ind]) > np.max(np.array(cut_ch)[s1+j+1][y_ind]):
                        all_ind_j.append(y_ind) 

            cut_ch[s1+j+1] = np.delete(cut_ch[s1+j+1], all_ind_j,0)
            p_all[s1+j+1] = np.delete(p_all[s1+j+1], all_ind_j,0) 
            all_ind_j=[]
print("--- %s seconds ---" % (time.time() - start_time))            

In [ ]:
# Sort the detected spikes using GEMsort without data reduction (to know the number of spikes in each group)

channels_num = ch_num
GEMsort_cluster = []
#GEMsort_cluster_number=[]
GEMsort_clusters_times = []

for i in range(channels_num): 
    GEMsort_cluster.append([])
    #GEMsort_cluster_number.append([])
    GEMsort_clusters_times.append([])
    
GEMsort_ave_cluster = []
for i in range(channels_num): 
    GEMsort_ave_cluster.append([])    

In [ ]:
start_time = time.time()
ch0 = 0 
channels_num = 384

for ch in range(ch0, channels_num): #for every channel 
    if ch not in ch_ref:
#     print(ch)
        res = np.array((cut_ch)[ch])   
        varcovmat = np.cov(res.T)
        k = np.array(np.mean(res.T, 1))
        tt = []
        for s in range(res.shape[0]):
            tt.append(k)
        varcovmat = res.T - np.array(tt).T

        u, s, v = svd(varcovmat)
        pca_data = np.dot(res,u[:, 0:2])

        GNG_dataset = (pca_data / 500.)
        prenumnode = round(pca_data.shape[0] / 50)  
        prenumnode_main = prenumnode
        GNG_eb = 0.9  # Learning rate of the winning node S 
        GNG_en = 0.0001  # Learning rate of the direct topological neighbors of S 
        GNG_edgethred = 10
        GNG_beta = 0.001
        GNG_alpha = 0.5
        Nodes,GNG_C,GNG_w, nodes_total,prenumnode,d,nodess = GEMsort(prenumnode, GNG_dataset,GNG_eb,GNG_en,GNG_edgethred,GNG_beta,GNG_alpha)

        #determinig number of groups
        all_nodes = []
        set1 = []
        set2 = []
        if np.array(Nodes).shape[0] != 0:
            for i in range(prenumnode):
                for j in range(prenumnode):
                    if GNG_C[i, j] != -1.:
                        set1.append(i)
                        set2.append(j)
            d = np.zeros((np.array(set1).shape[0], 2))
            d[:, 0] = set1
            d[:, 1] = set2
            for k in range(prenumnode):
                nodes = []
                for i in range(d.shape[0]):
                    nodes.append(k)
                    if d[i, 0] == k:
                        nodes.append(d[i, 1])
                    # print nodes
                for i in range(d.shape[0]):
                    for j in nodes:
                        if d[i, 0] == j:
                            nodes.append(d[i, 1])
                for i in range(d.shape[0]):
                    if d[i, 1] == k:
                        nodes.append(d[i, 0])
                for i in range(d.shape[0]):
                    for j in (nodes):
                        if d[i, 1] == j:
                            nodes.append(d[i, 0])
                nodes = np.unique(nodes)
                all_nodes.append(nodes)

            #identifying final groups
            nodes_total = []
            for k in range(prenumnode):
                a = []
                for j in range(prenumnode):
                    if np.array(all_nodes)[j][-1] == k:
                        a = np.array(all_nodes)[j]
                if a!= []:
                    nodes_total.append(a)
            nodes_total 

            #put the clusters with common nodes in the same group
            for i in range(np.array(nodes_total).shape[0]):
                if i < np.array(nodes_total).shape[0]:
                    for j in range(i): 
                        if i < np.array(nodes_total).shape[0]:
        #                     print(np.array(nodes_total)[i])
        #                     print(np.array(nodes_total)[j])
                            a = np.where(np.array(nodes_total)[i] == np.array(nodes_total)[j])[0]
                            a_set = set(np.array(nodes_total)[i]) 
                            b_set = set(np.array(nodes_total)[j]) 

                            if (a_set & b_set): 
                                s = np.unique(np.concatenate((np.array(nodes_total)[i],np.array(nodes_total)[j])))
                                np.array(nodes_total)[i] = s
                                nodes_total = np.delete(nodes_total,j,0)
        #                         print(nodes_total)

            line_nodess = np.hstack(nodes_total)
            GNG_metric = 2.

        # have the members of every cluster
            for i in range(np.array(nodes_total).shape[0]):
                GEMsort_cluster[ch].append([])
                GEMsort_clusters_times[ch].append([])

            for i in range(GNG_dataset.shape[0]): 
                dis = []
                for j in range(len(line_nodess)): #number of groups           
                        dis.append((np.linalg.norm((GNG_dataset[i] - GNG_w[int(line_nodess[j])]), GNG_metric) ** 2))
                mindis, minarg = np.array(dis).min(), np.array(dis).argmin()
                near_node = line_nodess[minarg]

                for j in range(len(nodes_total)):
                    for k in range(len(nodes_total[j])):
                        if nodes_total[j][k] == line_nodess[minarg]:
                            GEMsort_cluster[ch][j].append(res[i])    
                            #GNG_dataset2[i,2] = j
                            GEMsort_clusters_times[ch][j].append(GEMsort_all_times[ch][i]) 
        #                     print('ch=%d' %ch)
        #                     print(GEMsort_clusters_times[ch][j])
            #keep the average of clusters in a matrix
            for i in range(np.array(GEMsort_cluster[ch]).shape[0]):
                GEMsort_ave_cluster[ch].append([])   

            for k in range(np.array(GEMsort_cluster[ch]).shape[0]): 
                GEMsort_ave_cluster[ch][k] = np.mean(GEMsort_cluster[ch][k],axis=0)
print("--- %s seconds ---" % (time.time() - start_time))       

#plot spikes in every cluster
# ch = 86
# for i in range(np.array(GEMsort_cluster[ch]).shape[0]):
#     for k in range(np.array(GEMsort_cluster[ch][i]).shape[0]):
#          plt.plot(GEMsort_cluster[ch][i][k])
#     plt.show() 
        

In [ ]:
# know the ids (numbers) for clusters: managing the ids channel by channel 
num_id = -1
GEMsort_clust_id = []
GEMsort_all_id_one_row = []
for ch in range (channels_num):
    GEMsort_clust_id.append([])
    
for ch in range (channels_num):
        #print(ch)
        for j in range((np.array(GEMsort_cluster[ch]).shape[0])):
            #print(np.array(GEMsort_cluster[ch]).shape[0])  
            num_id = num_id + 1
            GEMsort_clust_id[ch].append(num_id)

# copy arrays        
GEMsort_clusters_times_original = copy.deepcopy(GEMsort_clusters_times)
GEMsort_cluster_original = copy.deepcopy(GEMsort_cluster) #this contains spikes of channels
GEMsort_clust_id_original = copy.deepcopy(GEMsort_clust_id)

        

In [ ]:
# spikes times detected by GEMsort

GEMsort_all_times_num_GEMsort = 0
i = 0
for k in range(384):
    #      if (a.shape[0]) < 1:
    #         #print(k)
            i = i + 1 #number of channels with spikes
            GEMsort_all_times_num_GEMsort = GEMsort_all_times_num_GEMsort + np.array(GEMsort_all_times)[k].shape[0]
            #print(np.array(GEMsort_all_times)[k].shape[0])
print(GEMsort_all_times_num_GEMsort) 


In [ ]:
#this is the number of clusters before merging
GEMsort_all_id_one_row = []
for ch in range(384):
    if i not in ch_ref:
        if np.array(GEMsort_clust_id[ch]).shape[0] > 0: 
            for idd in range(np.array(GEMsort_clust_id[ch]).shape[0]): 
                GEMsort_all_id_one_row.append(GEMsort_clust_id[ch][idd]) 

GEMsort_all_id_one_row = np.array(np.unique(GEMsort_all_id_one_row))    


In [ ]:
#merging similar clusters coming from near channels: for every ch_near near channels

ch_near = 30 #number of channels to be considered as near (depends on electrode characteristics)


corr_thr1 = 0.5
for ch in range(ch_near, 384 - ch_near): 
    if ch not in ch_ref:
#     print(ch)
        for i in range(-ch_near,ch_near):
            if (np.array(GEMsort_clust_id[ch]).shape[0] > 0) & (np.array(GEMsort_clust_id[ch+i+1]).shape[0]) > 0: #it is not null
                for p in range(np.array(GEMsort_clust_id[ch]).shape[0]):
                    corr = []
                    indexes = []
                    for t in range(np.array(GEMsort_clust_id[ch+i+1]).shape[0]):
                        ave_main = np.mean(GEMsort_cluster[ch][p],axis=0)
                        ave_nonmain = np.mean(GEMsort_cluster[ch+i+1][t],axis=0)
                        indexes.append(p) #just to choose the highest corr
                        corr.append(1 - spatial.distance.cosine(ave_nonmain, ave_main))       
        #                 corr_max[j]=max((np.array(corr)))
        #                 index_max[j] = np.argmax((np.array(corr)))
                    corr_final = max((np.array(corr))) 
                    sj = np.argmax((np.array(corr))) 

                    if corr_final > corr_thr1:
    #                     print('GEMsort_clust_id[ch+i+1][sj]', GEMsort_clust_id[ch+i+1][sj])
                        GEMsort_clust_id[ch+i+1][sj] = GEMsort_clust_id[ch][p]
#                     print('GEMsort_clust_id[ch+i+1][sj]', GEMsort_clust_id[ch+i+1][sj])



In [ ]:

for ch in range(382): 
#     print(ch)
    if (np.array(GEMsort_clust_id[ch]).shape[0] > 0) & (np.array(GEMsort_clust_id[ch+1]).shape[0]) > 0: #it is not null
    
        for p in range(np.array(GEMsort_clust_id[ch]).shape[0]):
            corr = []
            indexes = []
            for t in range(np.array(GEMsort_clust_id[ch+1]).shape[0]):
                ave_main = np.mean(GEMsort_cluster[ch][p],axis=0)
                ave_nonmain = np.mean(GEMsort_cluster[ch+1][t],axis=0)
                indexes.append(p) #to choose the highest corr
                corr.append(1 - spatial.distance.cosine(ave_nonmain, ave_main))       
                #corr_max[j] = max((np.array(corr)))
                #index_max[j] = np.argmax((np.array(corr)))
                #print('corr',corr)
            corr_final = max((np.array(corr))) 
            sj = np.argmax((np.array(corr))) 
            #ss=index_max[sj] 
        
            if corr_final > corr_thr1:
                GEMsort_clust_id[ch+1][sj] = GEMsort_clust_id[ch][p]
                
       #for next channel ch+2
            if (np.array(GEMsort_clust_id[ch+2]).shape[0]>0):
                corr = []
                indexes = []
                for t in range(np.array(GEMsort_clust_id[ch+2]).shape[0]):
                    ave_main = np.mean(GEMsort_cluster[ch][p],axis=0)
                    ave_nonmain = np.mean(GEMsort_cluster[ch+2][t],axis=0)
                    indexes.append(p) 
                    corr.append(1 -spatial.distance.cosine(ave_nonmain, ave_main))       
                    #corr_max[j]=max((np.array(corr)))
                    #index_max[j] = np.argmax((np.array(corr)))
                    #print('corr',corr)
                corr_final = max((np.array(corr))) 
                sj = np.argmax((np.array(corr))) 
                #ss=index_max[sj]

                if corr_final > corr_thr1:
                    GEMsort_clust_id[ch+2][sj] = GEMsort_clust_id[ch][p]
                    
          #for next channel ch+3
            if (np.array(GEMsort_clust_id[ch+3]).shape[0] > 0):
                corr = []
                indexes = []
                for t in range(np.array(GEMsort_clust_id[ch+3]).shape[0]):
                    ave_main = np.mean(GEMsort_cluster[ch][p],axis=0)
                    ave_nonmain = np.mean(GEMsort_cluster[ch+3][t],axis=0)
                    indexes.append(p) #to choose the highest corr
                    corr.append(1 - spatial.distance.cosine(ave_nonmain, ave_main))       
                    #corr_max[j]=max((np.array(corr)))
                    #index_max[j] = np.argmax((np.array(corr)))
                    #print('corr',corr)
                corr_final = max((np.array(corr))) 
                sj = np.argmax((np.array(corr)))

                if corr_final > corr_thr1:
                    GEMsort_clust_id[ch+3][sj] = GEMsort_clust_id[ch][p]
                    
#            #for next channel ch+4
#             if (np.array(GEMsort_clust_id[ch+4]).shape[0] > 0):
#                 corr = []
#                 indexes = []
#                 for t in range(np.array(GEMsort_clust_id[ch+4]).shape[0]):
#                     ave_main = np.mean(GEMsort_cluster[ch][p],axis=0)
#                     ave_nonmain = np.mean(GEMsort_cluster[ch+4][t],axis=0)
#                     indexes.append(p) #just to choose the highest corr
#                     corr.append(1 - spatial.distance.cosine(ave_nonmain, ave_main))       
#     #                 corr_max[j] = max((np.array(corr)))
#     #                 index_max[j] = np.argmax((np.array(corr)))
#                 print('corr',corr)
#                 corr_final = max((np.array(corr))) 
#                 sj = np.argmax((np.array(corr))) 
#                 #ss = index_max[sj] 

#                 if corr_final > corr_thr1:
#                     GEMsort_clust_id[ch+4][sj] = GEMsort_clust_id[ch][p]
                    
#             #for next channel ch+5
#             if (np.array(GEMsort_clust_id[ch+5]).shape[0] > 0):
#                 corr = []
#                 indexes = []
#                 for t in range(np.array(GEMsort_clust_id[ch+5]).shape[0]):
#                     ave_main = np.mean(GEMsort_cluster[ch][p],axis=0)
#                     ave_nonmain = np.mean(GEMsort_cluster[ch+5][t],axis=0)
#                     indexes.append(p) # to choose the highest corr
#                     corr.append(1 - spatial.distance.cosine(ave_nonmain, ave_main))       
#     #                 corr_max[j] = max((np.array(corr)))
#     #                 index_max[j] = np.argmax((np.array(corr)))
#                 print('corr',corr)
#                 corr_final = max((np.array(corr))) 
#                 sj = np.argmax((np.array(corr))) 
#                 #ss = index_max[sj]

#                 if corr_final > corr_thr1:
#                     GEMsort_clust_id[ch+5][sj] = GEMsort_clust_id[ch][p] 
                    
#             #for next channel ch+6
#             if (np.array(GEMsort_clust_id[ch+6]).shape[0] > 0):
#                 corr = []
#                 indexes = []
#                 for t in range(np.array(GEMsort_clust_id[ch+6]).shape[0]):
#                     ave_main = np.mean(GEMsort_cluster[ch][p],axis=0)
#                     ave_nonmain = np.mean(GEMsort_cluster[ch+6][t],axis=0)
#                     indexes.append(p) #just to choose the highest corr
#                     corr.append(1 -spatial.distance.cosine(ave_nonmain, ave_main))       
#     #                 corr_max[j] = max((np.array(corr)))
#     #                 index_max[j] = np.argmax((np.array(corr)))
#                 print('corr',corr)
#                 corr_final = max((np.array(corr))) 
#                 sj = np.argmax((np.array(corr))) 
#                 #ss = index_max[sj] 

#                 if corr_final > corr_thr1:
#                     GEMsort_clust_id[ch+5][sj] = GEMsort_clust_id[ch][p]
        

In [ ]:
#this is the number of clusters after merging
GEMsort_all_id_one_row = []
for ch in range(384):
    if i not in ch_ref:
        if np.array(GEMsort_clust_id[ch]).shape[0] > 0: 
            for idd in range(np.array(GEMsort_clust_id[ch]).shape[0]): 
                GEMsort_all_id_one_row.append(GEMsort_clust_id[ch][idd]) #for the comparison matrix later only

GEMsort_all_id_one_row = np.array(np.unique(GEMsort_all_id_one_row))    
# GEMsort_all_id_one_row.shape 

# Results: